In [ ]:
pip install -U datasets fsspec pytorch-lightning tensorboard albumentations torchmetrics

In [ ]:
import os
import sys
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from google.colab import drive

from datasets import load_dataset

In [ ]:
project_path = ""

sys.path.append(project_path)

In [ ]:
from module import DenseNetClassifierBinary
from datamodule import BrainTumorDataModule
from callbacks import get_callbacks
from helper import set_seed

In [ ]:
%load_ext tensorboard

In [ ]:
drive.mount('/content/drive')

In [ ]:
set_seed(42)

In [ ]:
# Load brain tumor dataset from Hugging Face (auto-cached locally)
ds = load_dataset("Cayanaaa/BrainTumorDatasets", name="binary")

In [ ]:
# Display class labels and their corresponding integer indices
print(ds['train'].features['label'].names)

In [ ]:
# Split Data
train_data = ds['train']
images = train_data['image']
labels = train_data['label']

In [ ]:
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels,
                                                                  test_size=0.2,
                                                                  stratify=labels,
                                                                  random_state=42
                                                                  )

In [ ]:
data_module = BrainTumorDataModule(
    train_data = (train_imgs, train_labels),
    val_data = (val_imgs, val_labels),
    batch_size = 64,
    img_size = (224, 224),
    num_workers = 4
)

In [ ]:
model = DenseNetClassifierBinary(
    learning_rate = 1e-3,
    weight_decay = 1e-5
)

In [ ]:
checkpoint_cb = ""

os.makedir(checkpoint_cb, exist_ok=True)

callbacks_v = get_callbacks(
    monitor_v = 'val_loss',
    mode_v = 'min',
    patience_v = 3
)

In [ ]:
logger_v = TensorboardLogger(
    save_dir='logs',
)

In [ ]:
trainer = pl.trainer(
    max_epochs = 100,
    accelerator = 'auto',
    callbacks = callbacks_v,
    logger = logger_v,

)